# Implementing the kNN algorithm for classification
<hr/>

## 1 Introduction
### 1.1 About the algorithm

The k-nearest neighbours (kNN) algorithm is a very simple, easy to understand and easy to implement algorithm. It is an algorithm wherein the <b>output is dependent on the <i>k</i> closest neighbors</b> of a given dataset. kNN can be used for either classification or regression.

* In the case of classification, the aim is to find the class of a given input data; the output being the class label it belongs to, which is the most common class label amongst <i>k</i> of its nearest neighbours.
* In the case of regression, the output is the average value of an attribute to the <i>k</i> nearest neighbours.

### 1.2 Lazy Algorithm

The kNN algorithm is sometimes reffered as a "Lazy Learning" algorithm, as oppose to the "Eager Learning" algorithm. The lazy learning method is a method in which the training set is only memorized and no descriminatory functions are involved. The eager learning method is thus the method wherein the model tries to find a generalisation.



## 2 The dataset
### 2.1 About the dataset

The dataset used is "The Iris dataset" which is a very common dataset used for training and testing machine learning models. This data is stored in a CSV file of 150 rows. The dataset is used to classify the Iris flower based on 4 attributes:

1. Sepal length
2. Sepal width
3. Petal length
4. Petal width

Using these attributes, the flower is categorized into one of following class:

1. Setosa
2. Versicolor
3. Virginica

### 2.2 Viewing the dataset

In [1]:
! head iris.csv

sepal_length,sepal_width,petal_length,petal_width,species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


It can be see that the dataset consists of 5 columns, the last one being the class the flower belongs to. It should also be noted that the first line consists of the column names, which needs to be removed when taking in the data.

## 3. Understanding the logic

### 3.1 Modus operandi

To find the label of a given data point, first the distances from that point to all the other points are found out, and the <i>k</i> nearest ones are selected. Another function determines the label by finding out which is the most common label amongst the <i>k</i> selected nearest neighbours.

### 3.2 Euclidean and Manhattan distance measures

There are many different ways to find the distance between two points. The most commonly used methods are the <b>Euclidean measure</b> and the <b>Manhattan measure</b>.
    
#### 3.2.1 Euclidean measure

The Euclidean measure considers the distance between two points as <b>the length of the straight line</b> passing through both points.

![Euclidean Distance from Wikipedia](https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0)

#### 3.2.2 Manhattan measure

In the Manhattan measure, the distance between two points is taken as <b>the sum of the absolute difference</b> of the coordinates.

![Taxicab geometry from Wikipedia](https://wikimedia.org/api/rest_v1/media/math/render/svg/02436c34fc9562eb170e2e2cfddbb3303075b28e)

## 4. Implementing the program

### 4.1 Importing the required modules
The following modules are required to read the CSV file, for math operations and for spliting the dataset into training and testing sets.

In [2]:
import csv
import math
import random

### 4.2 Loading the data

The data in the CSV file needs to be loaded into the list. The first line needs to be omitted because it just consists strings of the column names. And since the CSV reader function loads the data as strings, all the numerical data needs to be converted to floats. The loadCSV() function handles all this.

In [3]:
def loadCSV():
    data= csv.reader(open(r'iris.csv'))
    lines = list(data)
    lines = lines[1:]
    dataSize = len(lines[0])
    
    for i in range(len(lines)):
        for j in range(len(lines[i])):
            if j == len(lines[i]) - 1:
                lines[i][j] = str(lines[i][j])
            else:
                lines[i][j] = float(lines[i][j])
        
    return lines

### 4.3 Spliting the data into training and testing sets

The loaded data needs to be divided into testing and training sets. This partitioning is determined by the 'splitRatio' variable. Generally a ratio of 67:33 training to testing is taken. The data is chosen randomly from the dataset.

In [4]:
def splitData(data, splitRatio = 0.67):
    trainSize = int(len(data) * splitRatio)
    trainSet = []
    testSet = list(data)
    
    while len(trainSet) < trainSize:
        index = random.randrange(len(testSet))
        trainSet.append(testSet.pop(index))
        
    random.shuffle(testSet)
        
    return [trainSet, testSet]

### 4.4 Locating the nearest <i>k</i> neighbours

The nearest <i>k</i> neighbours can be found out by finding the distance between the input vector and the distance between all the points, and picking the <i>k</i> points with the shortest distance. The distance is found out by using the Euclidean Distance metric. 

In [5]:
def euclideanDist(inData1, inData2):
    distance = 0

    for i in range(len(inData1)-1):
        distance += math.pow(inData1[i] - inData2[i], 2)
        
    return math.sqrt(distance)

In [6]:
def getNeighbours(trainSet, testVector, k = 10):
    neighbours = []
    
    for point in trainSet:
        dist = euclideanDist(point, testVector)
        neighbours.append([point, dist])
        
    neighbours.sort(key = lambda x: x[1])
    
    nearNeighbours = []
    for i in range(k):
        nearNeighbours.append(neighbours[i][0])
        
    return nearNeighbours

### 4.5 Predictions and Accuracy

Predictions are made by looking at the labels of the nearest datasets and labelling the test data point the class which is the most common. The getPredict() function calls the getInstancePredict() for each of the point in the test set, which all gets alloted a label. The list of this label is returned.

The accuracy is determined by getting the number of correct predictions.

In [7]:
def getInstancePredict(nearN, testV):
    classCount = {}
    
    for vertex in nearN:
        if vertex[-1] not in classCount:
            classCount[vertex[-1]] = 1
        else:
            classCount[vertex[-1]] += 1
            
    bestLabel, maxCount = None, -1
    
    for x in classCount:
        if bestLabel == None or classCount[x] > maxCount:
            bestLabel = x
            maxCount = classCount[x]
            
    return bestLabel

In [8]:
def getPredict(trainSet, testSet):
    predictions = []
    
    for testValue in testSet:
        nearN = getNeighbours(trainSet, testValue)
        predictions.append(getInstancePredict(nearN, testValue))
        
    return predictions

In [9]:
def getAccuracy(pred, testSet):
    count = 0
    
    for i in range(len(testSet)):
        if pred[i] == testSet[i][-1]:
            count += 1
            
    return (count/float(len(testSet)))*100.0

### 4.6 The main() function

The main function encapsulates all of the above functions.

In [10]:
def main():
    data = loadCSV()
    trainSet, testSet = splitData(data, 0.66)

    print("{} rows split into {} training and {} testing sets".format(len(data), len(trainSet), len(testSet)))

    pred = getPredict(trainSet, testSet)
    accuracy = getAccuracy(pred, testSet)

    print("Accuracy : {0:.3f}%".format(accuracy))
    
main()

150 rows split into 99 training and 51 testing sets
Accuracy : 98.039%


## 5. References

1. [Youtube : KNN Algorithm using Python](https://www.youtube.com/watch?v=6kZ-OPLNcgE)
2. [Wikipedia : k-nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)
3. [Wikipedia : Euclidean Distance](https://en.wikipedia.org/wiki/Euclidean_distance)
4. [Wikipedia : Manhattan Distance](https://en.wikipedia.org/wiki/Taxicab_geometry)
5. [CSV File : The Iris Dataset ](https://gist.github.com/a08a1080b88344b0c8a7.git)
5. [Wikipedia : Iris flower data set](https://en.wikipedia.org/wiki/Iris_flower_data_set)